# IBBI Package Quickstart (Google Colab)

This notebook provides a 0-setup way to test the core features of the `ibbi` package using a free Google Colab GPU.

We will walk through:
1.  Installing `ibbi`
2.  Listing available models
3.  Loading a pre-trained model
4.  Running inference (prediction)
5.  Running evaluation on a subset of data
6.  Running model explainability (LIME)

To use a GPU for this notebook, go to **Runtime > Change runtime type** and select **T4 GPU**.

## 1. Install `ibbi` Package

In [ ]:
# --- 1. Install IBBI and Dependencies for Colab ---

# First, install the specific dependencies that ibbi needs but
# Colab doesn't provide by default.
# We specify the exact ultralytics version from the ibbi package
!pip install ultralytics==8.3.139 hf-xet slicer lime scikit-bio

# Now, install ibbi *without* its dependencies.
# This forces ibbi to use the packages already in Colab (like torch,
# numpy, pandas) and the ones we just installed, which prevents
# the dependency conflicts you saw in the log.
!pip install --no-dependencies ibbi

# Print the installed ibbi version
!python -c "import ibbi; print(f'ibbi version installed: {ibbi.__version__}')"

## 2. List Available Models

In [ ]:
import ibbi

print("Available models in the `ibbi` package:")
# 'as_df=True' returns a pandas DataFrame, which Colab displays nicely.
ibbi.list_models(as_df=True)

## 3. Load Data and a Pre-trained Model

We'll use the convenient 'species_classifier' alias to load the recommended model for multi-class detection. We'll also load the built-in test dataset.

In [ ]:
from PIL import ImageDraw, ImageFont

# Load the model. The first time, this will download the model weights.
model = ibbi.create_model("species_classifier", pretrained=True)

# Load the built-in test dataset
test_dataset = ibbi.get_dataset()

# Select a single image to run inference on
image = test_dataset[1010]["image"]

print("Model and data loaded successfully.")

## 4. Run Inference (Prediction)

Now we'll run the model on our sample image. We'll also draw the bounding boxes to visualize the result.

In [ ]:
# Run prediction
results = model.predict(image)

# --- Visualize the results ---
image_with_boxes = image.copy()
draw = ImageDraw.Draw(image_with_boxes)

try:
    # Colab has default fonts, but we'll try to load a clear one
    font = ImageFont.truetype("Arial.ttf", 20)
except OSError:
    font = ImageFont.load_default()

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    text = f"{label}: {score:.2f}"
    draw.rectangle(box, outline="darkblue", width=3)
    draw.text((box[0] + 5, box[1] + 5), text, fill="white", font=font)

print("Prediction results:", results)

# By making the PIL Image the last line of the cell, it will display automatically.
image_with_boxes

## 5. Run Model Evaluation

We'll run the evaluator on a **small subset** of the test data. Running on the full dataset might cause Colab to run out of RAM.

In [ ]:
import numpy as np

# Create a small 10-sample subset for evaluation to avoid RAM issues
# We use .select(), not Python slicing
data_subset = test_dataset.select(range(10))

# Initialize the evaluator
evaluator = ibbi.Evaluator(model)

# Run classification and detection performance analysis
performance = evaluator.object_classification(data_subset, iou_thresholds=np.arange(0.5, 1.0, 0.05))

print(f"Evaluation complete on {len(data_subset)} images.")
print(f"Overall mAP: {performance['mAP']:.4f}")

## 6. Run Model Explainability (LIME)

Finally, we can use the `Explainer` to see *why* the model made a prediction.

In [ ]:
# Initialize the explainer
explainer = ibbi.Explainer(model)

# Generate a LIME explanation (this is fast)
lime_explanation, original_image = explainer.with_lime(image)

# Plot the explanation
ibbi.plot_lime_explanation(lime_explanation, original_image, top_k=1)